# 📊 Optimización de Hiperparámetros con GridSearchCV

Este notebook realiza el modelado predictivo para la **cancelación de clientes (churn)** usando Random Forest, optimizando los hiperparámetros con **GridSearchCV**.  
Partimos desde el dataset limpio generado en `01_EDA.ipynb`.

### 📌 ¿Qué hace GridSearchCV?

- Prueba múltiples combinaciones de valores para varios hiperparámetros.
- Utiliza validación cruzada (Cross-Validation) para evaluar cada configuración de forma robusta.
- Retorna la mejor configuración y el modelo entrenado con estos hiperparámetros.

⚙️ El flujo es:
- Cargar los datos limpios.
- Separar la variable objetivo y las características predictoras.
- Codificar las variables categóricas.
- Dividir el dataset en entrenamiento y prueba.
- Configurar un modelo **Random Forest**.
- Optimizar hiperparámetros usando **GridSearchCV**.
- Evaluar el desempeño del mejor modelo.

---


# 1️⃣ 📚 Importar librerías necesarias

Primero importamos todas las librerías necesarias para la modelización, la optimización y la evaluación del modelo.

Importamos los paquetes clave para:
- Manipular datos (`pandas`, `numpy`)
- Crear y evaluar el modelo (`sklearn`)

In [57]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


# 2️⃣ Cargar datos limpios 📑

Importamos nuevamente el dataset limpio que preparamos en el EDA (`01_EDA.ipynb`).

Este archivo ya no contiene columnas irrelevantes como `customerID` y tiene variables limpias listas para usar.

In [58]:
# Cargar dataset limpio y enriquecido
df = pd.read_csv('../data/processed/telco_churn_featured.csv')
print("Tamaño del dataset:", df.shape)
df.head()



Tamaño del dataset: (7010, 25)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,AvgCharges,TenureGroup,HasPhoneService,MultipleLinesBinary,TotalServices
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,Yes,Electronic check,29.85,29.85,No,29.850000,0–12,0,0,1
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,No,Mailed check,56.95,1889.50,No,55.573529,24–48,1,0,3
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,Yes,Mailed check,53.85,108.15,Yes,54.075000,0–12,1,0,3
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,No,Bank transfer (automatic),42.30,1840.75,No,40.905556,24–48,0,0,3
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,Yes,Electronic check,70.70,151.65,Yes,75.825000,0–12,1,0,1


# 3️⃣  Definir variable objetivo (y) y características predictoras (X) 🎯

- `y` será la columna **Churn** (cancelación o no).
- `X` son todas las demás columnas (excluimos `Churn`).

Esto define qué queremos predecir y qué variables usaremos como entrada para entrenar el modelo.


In [59]:
y = df['Churn']
X = df.drop('Churn', axis=1)

print("Forma de X:", X.shape)
print("Forma de y:", y.shape)


Forma de X: (7010, 24)
Forma de y: (7010,)


🔹 Forma de X: (7010, 24)

7010 = cantidad de observaciones o filas.

➤ Representa el número de clientes en el dataset.

24 = cantidad de características o columnas.

➤ Significa que cada cliente tiene 24 variables predictoras (por ejemplo: tipo de contrato, método de pago, servicios contratados, etc.).

Esto indica que tienes un dataset con 7010 clientes y 24 atributos por cliente que serán usados para predecir su comportamiento.

🔹 Forma de y: (7010,)

Esta es la forma del vector y, que contiene únicamente la variable objetivo Churn.

El valor (7010,) indica que es un vector unidimensional con 7010 elementos, uno por cada cliente.

# 4️⃣  Codificar variables categóricas 🔑

Los algoritmos de Machine Learning necesitan datos numéricos.  

Aquí:
- Identificamos qué columnas son categóricas.
- Aplicamos **One-Hot Encoding** para convertirlas en variables dummy (0 o 1).
- Usamos `drop_first=True` para evitar multicolinealidad.


In [60]:
# Detectar columnas categóricas
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
print("Columnas categóricas:", cat_cols)

Columnas categóricas: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TenureGroup']


In [61]:
# One-Hot Encoding
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)
print("Forma después de codificar:", X_encoded.shape)

X_encoded.head()

Forma después de codificar: (7010, 38)


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,AvgCharges,HasPhoneService,MultipleLinesBinary,TotalServices,gender_Male,Partner_Yes,...,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TenureGroup_12–24,TenureGroup_24–48,TenureGroup_48–60,TenureGroup_60+
0,0,1,29.85,29.85,29.850000,0,0,1,False,True,...,False,False,True,False,True,False,False,False,False,False
1,0,34,56.95,1889.50,55.573529,1,0,3,True,False,...,True,False,False,False,False,True,False,True,False,False
2,0,2,53.85,108.15,54.075000,1,0,3,True,False,...,False,False,True,False,False,True,False,False,False,False
3,0,45,42.30,1840.75,40.905556,0,0,3,True,False,...,True,False,False,False,False,False,False,True,False,False
4,0,2,70.70,151.65,75.825000,1,0,1,False,False,...,False,False,True,False,True,False,False,False,False,False


| Elemento | Valor    | Significado                                                         |
| -------- | -------- | ------------------------------------------------------------------- |
| **7010** | filas    | Hay **7010 clientes**, igual que antes                              |
| **38**   | columnas | Ahora hay **38 variables numéricas**, en lugar de las 24 originales |

Esto significa que al codificar las variables categóricas en columnas binarias, el total de columnas aumentó de 24 a 38 columnas numéricas.

# 5️⃣  Dividir el dataset en entrenamiento y prueba ✂️

- Usamos 80% de los datos para entrenar y 20% para validar.
- Estratificamos según `y` para mantener la proporción de clases.


In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)


X_train: (5608, 38)
X_test: (1402, 38)


test_size=0.2 → quiere decir que el 20% de los datos se va al conjunto de prueba.

Entonces:

20% de 7010 = 1402 → datos de prueba

80% de 7010 = 5608 → datos de entrenamiento

| Conjunto  | Tamaño     | Uso               | Contenido                                |
| --------- | ---------- | ----------------- | ---------------------------------------- |
| `X_train` | (5608, 38) | Entrenamiento     | 5608 registros, 38 variables predictoras |
| `X_test`  | (1402, 38) | Evaluación (test) | 1402 registros, 38 variables predictoras |


# 6️⃣ Configurar modelo base y rejilla de hiperparámetros ⚙️

- Usaremos **Random Forest** como clasificador base.
- Definimos un **conjunto de combinaciones** de hiperparámetros para probar:
  - `n_estimators`: número de árboles.
  - `max_depth`: profundidad máxima de cada árbol.
  - `min_samples_split`: mínimo de muestras para dividir un nodo.


In [63]:
# Modelo base
rf = RandomForestClassifier(random_state=42)

In [64]:
# Rejilla de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

print("Rejilla de hiperparámetros:", param_grid)

Rejilla de hiperparámetros: {'n_estimators': [100, 200], 'max_depth': [None, 5, 10], 'min_samples_split': [2, 5]}


# 7️⃣ Configurar búsqueda con GridSearchCV 🔍

- `GridSearchCV` probará todas las combinaciones de la rejilla usando validación cruzada (5 folds).
- Busca la mejor combinación de hiperparámetros optimizando el `accuracy`.


In [65]:
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)


# 8️⃣  Ejecutar GridSearchCV 🚀

Este paso puede tardar unos minutos dependiendo de la cantidad de combinaciones y tamaño de los datos.


In [66]:
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


,estimator,RandomForestC...ndom_state=42)
,param_grid,"{'max_depth': [None, 5, ...], 'min_samples_split': [2, 5], 'n_estimators': [100, 200]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,200


# 9️⃣ Evaluar el mejor modelo ✅ 

Una vez completada la búsqueda:
- Revisamos los mejores hiperparámetros encontrados.
- Calculamos métricas sobre los datos de prueba:
  - `Accuracy`
  - Matriz de Confusión
  - Reporte de Clasificación (precision, recall, f1-score)


In [68]:
print("Mejores parámetros:", grid_search.best_params_)



Mejores parámetros: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}


In [69]:
print("Mejor accuracy en validación cruzada:", grid_search.best_score_)



Mejor accuracy en validación cruzada: 0.8011776472814411


In [70]:
y_pred = grid_search.predict(X_test)

print("Accuracy en test:", accuracy_score(y_test, y_pred))

Accuracy en test: 0.7960057061340942


In [71]:
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))


Matriz de Confusión:
 [[937  94]
 [192 179]]


|               | Predijo **No** (no canceló) | Predijo **Sí** (canceló) |
| ------------- | --------------------------- | ------------------------ |
| **Real: No**  | 937  (✔️ True Negative)     | 94   (❌ False Positive)  |
| **Real: Yes** | 192  (❌ False Negative)     | 179  (✔️ True Positive)  |


In [72]:
print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))

Reporte de Clasificación:
               precision    recall  f1-score   support

          No       0.83      0.91      0.87      1031
         Yes       0.66      0.48      0.56       371

    accuracy                           0.80      1402
   macro avg       0.74      0.70      0.71      1402
weighted avg       0.78      0.80      0.79      1402



Clase: No (cliente no canceló)
- Precision: 0.83 → De los que predije que no cancelaban, acerté el 83%
- Recall:    0.91 → De todos los que realmente no cancelaron, capturé el 91%
- F1-score:  0.87 → Balance entre precisión y recall

Clase: Yes (cliente canceló)
- Precision: 0.66 → De los que predije que cancelaban, acerté el 66%
- Recall:    0.48 → De todos los que realmente cancelaron, solo capturé el 48% 
- F1-score:  0.56 → El balance aquí es bajo


## 🎓 Conclusión del modelo Random Forest optimizado

El modelo entrenado con `RandomForestClassifier` y optimizado mediante `GridSearchCV` alcanzó una **precisión general (accuracy) del 80%** sobre el conjunto de prueba, lo cual indica un desempeño global sólido. No obstante, un análisis detallado por clase permite observar oportunidades claras de mejora:

### 📌 Desempeño por clase:

- **Clase 'No Churn' (Clientes que no cancelan):**
  - 🔹 **Precision:** 0.83
  - 🔹 **Recall:** 0.91
  - 🔹 **F1-Score:** 0.87  
  ✅ El modelo es muy efectivo al predecir correctamente a los clientes que permanecen con la compañía.

- **Clase 'Churn' (Clientes que cancelan):**
  - 🔸 **Precision:** 0.66
  - 🔸 **Recall:** 0.48
  - 🔸 **F1-Score:** 0.56  
  ⚠️ El modelo **subestima la cancelación de clientes**, detectando solo el 48% de los casos reales.

### 📉 Implicaciones:

Aunque el modelo predice bien a los clientes leales, **falla al identificar a tiempo a muchos clientes que van a cancelar el servicio**, lo que podría generar pérdidas económicas si no se detectan a tiempo.

---

## 🔧 Próximos pasos sugeridos

1. **Manejo del desbalance de clases:**
   - Probar técnicas como **SMOTE** para sobre-samplear la clase `Churn`.

2. **Explorar modelos más potentes:**
   - **XGBoost**, **LightGBM**, o **CatBoost**, que tienden a mejorar el recall de la clase minoritaria.

3. **Mejorar la ingeniería de variables:**
   - Incluir nuevas variables derivadas, como historial de atención al cliente, tiempo desde el último reclamo, o engagement mensual.

4. **Evaluaciones adicionales:**
   - Graficar **curvas ROC y AUC**.
   - Analizar la importancia de cada feature para explicar el comportamiento del modelo.

---

